In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import missingno as mo
from scipy.interpolate import interp1d
from scipy import stats
from scipy.stats import shapiro
# from OUTLIERS import smirnov_grubbs as grubbs

TODO:
- [ ] Add more visualizations (Sophie)
- [x] Missing values          (Amir)
  - [x] backfill for 1 day
  - [x] In case of long missing values, we still have to figure out what to do 
    - Most missing values after the day baqckfilling ended up being the most obscure appCat's so I simply set them to 0
  - [x] Aggregating over Days/Hours

- [ ] Outlier detection        (Sophie)
  - [ ] Grubs/ VIF/ Cook's distance (Not all of them)

- [ ] Feature engineering     (Nabila)
  - [ ] PCA                   
    - [ ] Whitening
    


In [ ]:
plt.rcParams.update({
    "font.family": "serif",
    "font.size": 18,
    "axes.titlesize": 18,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.figsize": (8, 6),
    "figure.dpi": 100,
    "savefig.dpi": 200,
    "savefig.format": "png",
    "savefig.transparent": True,
    "axes.grid": True,
    "grid.linewidth": 0.5,
    "grid.linestyle": "--",
    "grid.color": "0.8",
    "image.cmap": "Blues",
    "lines.linewidth": 1.5,
    "lines.markersize": 6,
    "text.usetex": True, "mathtext.fontset": "cm",
    "pgf.preamble": r"\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage{cmbright}"
})


In [ ]:
data = pd.read_csv("../data/raw/dataset_mood_smartphone.csv", index_col=0)
data['time'] = pd.to_datetime(data['time'])
print(data.tail())

In [ ]:
# Convert time column to datetime format
data['time'] = pd.to_datetime(data['time'])
data['time'] = data['time'].dt.round('H')


# Pivot the data to create separate columns for each variable
data_pivot = data.pivot_table(index=['id', 'time'], columns='variable', values='value')

# Reset the index to flatten the column hierarchy
data_pivot.reset_index(inplace=True)

In [ ]:
# Removing outliers from visual inspection

# Negative value from 'builtin' 
negative_builtin_indices = np.where(data_pivot['appCat.builtin'] < 0)
data_pivot = data_pivot.drop(negative_builtin_indices[0])

#Extreme positive value from 'entertainment' 
positive_ent_indices = np.where(data_pivot['appCat.builtin'] > 30000)
data_pivot = data_pivot.drop(positive_ent_indices[0])

#Extreme positive value from 'office' 
positive_office_indices = np.where(data_pivot['appCat.office'] > 30000)
data_pivot = data_pivot.drop(positive_office_indices[0])

#Extreme positive value from 'social' 
positive_social_indices = np.where(data_pivot['appCat.social'] > 30000)
data_pivot = data_pivot.drop(positive_social_indices[0])

In [ ]:
print(data_pivot.describe().to_markdown())

|       | time                          |     activity |   appCat.builtin |   appCat.communication |   appCat.entertainment |   appCat.finance |   appCat.game |   appCat.office |   appCat.other |   appCat.social |   appCat.travel |   appCat.unknown |   appCat.utilities |   appCat.weather |   call |   circumplex.arousal |   circumplex.valence |        mood |        screen |   sms |
|:------|:------------------------------|-------------:|-----------------:|-----------------------:|-----------------------:|-----------------:|--------------:|----------------:|---------------:|----------------:|----------------:|-----------------:|-------------------:|-----------------:|-------:|---------------------:|---------------------:|------------:|--------------:|------:|
| count | 27028                         | 27026        |      27027       |             27027      |              27026     |       26287      |    23775      |      26974      |     27025      |      27026      |      27022      |       27019      |         27027      |       25009      |  27022 |         27028        |         27028        | 27028       | 27026         | 27005 |
| mean  | 2014-04-14 16:34:06.144738816 |     0.120058 |         42.3545  |                45.6104 |                120.543 |          31.3167 |      356.289  |         45.7802 |        20.9953 |         89.0727 |         50.4012 |          42.3354 |            18.4875 |          29.719  |      1 |            -0.217589 |             0.668732 |     6.94341 |   113.027     |     1 |
| min   | 2014-02-17 07:00:00           |     0        |      -6343.29    |                 0.117  |                  0.005 |           1.003  |        1.003  |          0.044  |         0.014  |          0.552  |          0.08   |           0.111  |             0.976  |           1.007  |      1 |            -2        |            -2        |     1       |     0.0350001 |     1 |
| 25%   | 2014-04-02 06:45:00           |     0        |          4.41667 |                13.12   |                 10.794 |           8.014  |       80.6005 |         10.046  |         8.037  |         17.861  |         22.122  |          11.044  |             4.033  |           5.22   |      1 |            -1        |             0        |     6       |    19.334     |     1 |
| 50%   | 2014-04-15 15:00:00           |     0.025    |          8.02163 |                27.6707 |                 34.246 |          21.601  |      169.699  |         23.264  |        10.257  |         44.566  |         38.4282 |          21.185  |            10.537  |          10.147  |      1 |             0        |             1        |     7       |    44.771     |     1 |
| 75%   | 2014-04-27 22:00:00           |     0.158333 |         16.3092  |                51.6465 |                123.965 |          51.205  |      478.826  |         45.241  |        16.514  |        102.588  |         61.5086 |          30.178  |            20.0615 |          22.431  |      1 |             1        |             1        |     8       |    94.8592    |     1 |
| max   | 2014-06-09 00:00:00           |     1        |      19453       |              3133.48   |               2929.61  |         315.481  |     4115.3    |       4675.7    |      2243.24   |      15001.5    |       3492.32   |        2239.94   |           447.918  |         344.863  |      1 |             2        |             2        |    10       |  9539.82      |     1 |
| std   | nan                           |     0.190739 |        582.008   |                85.285  |                217.58  |          29.7814 |      634.744  |         80.0885 |        56.8702 |        161.532  |         76.5206 |         108.084  |            38.2665 |          70.1424 |      0 |             1.0681   |             0.686026 |     1.0813  |   338.403     |     0 |

In [ ]:
# fill in the screen time for the entire day, sum the number of sms and call for the entire day
data_pivot['day'] = data_pivot['time'].dt.floor('D')
data_pivot['screen'] = data_pivot['screen'].groupby([data_pivot['id'], data_pivot['day']]).transform('sum')
data_pivot['sms'] = data_pivot['sms'].groupby([data_pivot['id'], data_pivot['day']]).transform('sum')
data_pivot['call'] = data_pivot['call'].groupby([data_pivot['id'], data_pivot['day']]).transform('sum')

In [ ]:
# Visualize the number of missing data points
missing = data_pivot.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot(kind='bar', title='Number of missing data points')
plt.show()

In [ ]:

# Visualzing the missing data using a stacked bar plot over time
missing = data_pivot.isna().sum(axis=1)
missing = missing.groupby(data_pivot['time'].dt.floor('D')).sum()
missing.plot(kind='bar', title='Number of missing data points over time', figsize=(15, 5))
plt.xticks(np.arange(0, len(missing), 10), missing.index[::10].strftime('%Y-%m-%d'), rotation=45)
plt.title(f'Number of missing data points over time, total: {missing.sum()}')
plt.xlabel('Time')
plt.ylabel('Number of missing data points')
plt.show()

For the values here there a few missing points, we can use a sliding window to fill in the missing values
For the values where there are many value missing, these seem like varaibles where there they have mostly 0 values, so we can fill in the missing values with 0

In [ ]:
for column in data_pivot.columns:
    for user in data_pivot['id'].unique():
        # window sliding fill
        if 0 < data_pivot[column].isnull().sum() < 500:
            data_pivot[column][data_pivot['id'] == user] = data_pivot[column][data_pivot['id'] == user].rolling(4, min_periods=1).mean()
    

missing = data_pivot.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot(kind='bar', title='Number of missing data points')
plt.show()

# Aggregating over days

In [ ]:
day = data_pivot['time'].dt.floor('D')
data_pivot['day'] = day

day_data = data_pivot.groupby(['id', 'day']).mean()
day_data.reset_index(inplace=True)
# Sort data by id and day
day_data.sort_values(['id', 'day'], inplace=True)

Visualizing the missing days now

In [ ]:
missing = day_data.isnull().sum()
plt.bar(missing.index, missing)
plt.xticks(np.arange(0, len(missing), 1), missing.index, rotation=70)
plt.show()
print(f'Total days measured: {len(day_data)}, days with missing data {len(missing[missing > 0])}')

## Analysis of missing data

In [ ]:
mo.matrix(day_data)

In [ ]:
mo.heatmap(day_data)

In [ ]:
mo.dendrogram(day_data)

In [ ]:
for column in day_data.columns:
    if 'appCat.' in column:
        day_data[column].fillna(0, inplace=True)
        
day_data.dropna(subset=['mood'], inplace=True)

## Sort by id and time, to see how missing values relate between individuals

In [ ]:
mo.matrix(day_data)

In [ ]:
mo.heatmap(day_data)

In [ ]:
mo.dendrogram(day_data)

In [ ]:
# Make a matrix plot per id, for acitivity, circumplex arousal and valence and mood

fig, axes = plt.subplots(3,9, figsize=(30, 30))
axes = axes.flatten()
for i, id in enumerate(day_data['id'].unique()):
    id_data = day_data[day_data['id'] == id]
    id_data.set_index('time', inplace=True)
    id_data = id_data[['activity', 'circumplex.arousal', 'circumplex.valence', 'mood']]
    mo.matrix(id_data, ax=axes[i], figsize=(5, 5), sparkline=False)
    axes[i].set_title(f'ID: {id}')
    # Ensure there is a box around the plot
    axes[i].spines['top'].set_visible(True)
plt.tight_layout()

In [ ]:
print(len(day_data))

In [ ]:
# drop all rows where mood is missing
# Make a matrix plot per id, for acitivity, circumplex arousal and valence and mood

fig, axes = plt.subplots(3,9, figsize=(30, 30))
axes = axes.flatten()
for i, id in enumerate(day_data['id'].unique()):
    id_data = day_data[day_data['id'] == id]
    id_data.set_index('time', inplace=True)
    id_data = id_data[['activity', 'circumplex.arousal', 'circumplex.valence', 'mood']]
    mo.matrix(id_data, ax=axes[i], figsize=(5, 5), sparkline=False)
    axes[i].set_title(f'ID: {id}')
    # Ensure there is a box around the plot
    axes[i].spines['top'].set_visible(True)
plt.tight_layout()

In [ ]:
mo.heatmap(day_data)

In [ ]:
mo.dendrogram(day_data)

In [ ]:
for column in ['activity', 'circumplex.arousal', 'circumplex.valence']:
    print(f"Processing column: {column}")
    for user in day_data['id'].unique():
        user_data = day_data[day_data['id'] == user][column]
        
        # Perform linear interpolation
        interpolated_data = user_data.interpolate(method='linear')
        
        # Check if there are still NaNs and attempt to fill them
        if interpolated_data.isna().any():
            # Fill NaNs at the beginning and end by forward and backward filling
            interpolated_data.fillna(method='ffill', inplace=True)
            interpolated_data.fillna(method='bfill', inplace=True)

        # Assign back the interpolated (and potentially forward/backward filled) data
        day_data.loc[day_data['id'] == user, column] = interpolated_data

print(day_data.isnull().sum())

        


len(day_data)

# Conclusion
We can see that generally all data for each participant is complete, but with chucks missing, this might mean we can simply just drop the missing chunks of data, but this means that we can't predict someones mood for the first few days when they enter. It might be interesting to see if we can predict the mood even for these days using the other people's data and the data we have on the other stuff for that person, but it would require a special encoding of the data to differentiate between missing data and actual data. E.g. a value outside of the range of valid numbers.

In [ ]:
# Generating a toy data set to test model training

# Create a toy data set with 10 days of data for 10 users
np.random.seed(42)
users = np.arange(10)
days = pd.date_range('2014-02-01', periods=10, freq='D')
toy_data = pd.DataFrame(index=pd.MultiIndex.from_product([users, days], names=['id', 'time']))
# Give each user a random mean and std, use that to generate random data
for user in users:
    mean = np.random.uniform(0, 10)
    std = np.random.uniform(0, 5)
    for column in day_data.columns:
        if column in ['id', 'day', 'time']:
            continue
        else:
            toy_data.loc[user, column] = np.random.exponential(mean, size=10) / (day_data[column].max() - day_data[column].min()) + day_data[column].min()


In [ ]:
toy_data.reset_index(inplace=True)
toy_data.to_csv('../data/preprocessed/toy_data.csv', index=False)
toy_data.head()

In [ ]:
# get correlation matrix ignoring time and id

# numeric only
num_grouped = data_pivot.select_dtypes(include=[np.number])
# Drop call and sms variables
num_grouped = num_grouped.drop(columns=['call', 'sms'])
corr = num_grouped.corr()
plt.imshow(corr, cmap='Blues', interpolation='nearest')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.show()

## More Data Visualizations, i.e. stuff Sophie just added
It seems like something might be strange about the call and SMS data because they produce the exact same graph. 

In [ ]:
#Histograms for Call and SMS overall

sms_data = data_pivot["sms"]
call_data = data_pivot["call"]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].hist(sms_data, bins=30, color='blue', alpha=0.5)
axes[0].set_xlabel("Number of SMSs Sent")
axes[0].set_ylabel("Count")
axes[0].set_title('SMS')

axes[1].hist(call_data, bins=30, color='green', alpha=0.5)
axes[1].set_xlabel("Number of Calls Made")
axes[1].set_title('Call')

plt.tight_layout()
plt.show()

In [ ]:
#Bar plots for call and SMS over time

days = data_pivot['day']
sms_counts = data_pivot['sms']
call_counts = data_pivot['call']
time_sorted = sorted(days)
sorted_sms_counts = [sms_counts[index] for index in sorted(range(len(days)), key=lambda x: days[x])]
sorted_call_counts = [call_counts[index] for index in sorted(range(len(days)), key=lambda x: days[x])]
months = [date.strftime('%b') for date in time_sorted]

#SMS Data
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].bar(months, sorted_sms_counts, color='blue', alpha=0.5)
axes[0].set_ylabel('SMS Count')
axes[0].set_title('SMS Count Over Time')

#Call Data
axes[1].bar(months, sorted_call_counts, color='green', alpha=0.5)
axes[1].set_ylabel('Call Count')
axes[1].set_title('Call Count Over Time')


plt.tight_layout()
plt.show()


## Outlier Identification
The only non-normally distributed variable for the unbounded data is "screen". Grubb's tests for the normally distributed unbounded data all return none. Previous data cleaning must remove them.

Other methods, such as IQR and Z-score for identifying outliers in screen indicate that the outliers for screen are likely just 'natural outliers' and don't need to be removed from the data set.

In [ ]:
#Shapiro-Wilk Tests for normality to see if we can use Grubb's Test

variables = ['screen', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 
             'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 
             'appCat.utilities']

normality_results = {}

for variable in variables:
    data = data_pivot[variable]
    stat, p_value = shapiro(data)
    if p_value > 0.05:
        normality_results[variable] = 'Normally distributed'
    else:
        normality_results[variable] = 'Not normally distributed'

for variable, result in normality_results.items():
    print(f"Result for {variable}: {result}")

In [ ]:
#Trying some methods for identifying outliers. First, with the most basic using interquartile range.

def identify_outliers_IQR(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    upper_outliers = np.where(data >= upper_bound)[0]
    lower_outliers = np.where(data <= lower_bound)[0]
    return lower_outliers, upper_outliers

IQR_outliers_info = {}
for variable in variables: 
    IQR_outliers_info[variable] = identify_outliers_IQR(data_pivot[variable])
    #print(f"Outliers in {variable} found at indices: {outliers_info.get}")

for variable, result in IQR_outliers_info.items():
    if len(result[0] > 0) or len(result[1] > 0):
        print(f'Lower outliers in {variable} found at indices {result[0]}.')
        print(f'Upper outliers in {variable} found at indices {result[1]}.')
        print()     

In [ ]:
#Trying some methods for identifying outliers. First, with the most basic using interquartile range.

def identify_outliers_zScore(data):
    z = np.abs(stats.zscore(data))
    threshold_z = 2
    outlier_indices = np.where(z > threshold_z)[0]
    return outlier_indices

zScore_outliers_info = {}

for variable in variables: 
    zScore_outliers_info[variable] = identify_outliers_zScore(data_pivot[variable])

for variable, result in zScore_outliers_info.items():
    if len(result) > 0:
        print(f'Outliers in {variable} found at indices {result}.')

In [ ]:
#Plotting the Outliers
data_pivot = data_pivot.sort_values(by='time')
IQR_indices = np.concatenate((IQR_outliers_info['screen'][0],IQR_outliers_info['screen'][1]))
zScore_indices = zScore_outliers_info['screen']
common_indices = np.intersect1d(IQR_indices, zScore_indices)

plt.figure(figsize=(12, 6))
plt.scatter(data_pivot['time'], data_pivot['screen'], color = 'blue')
plt.scatter(data_pivot['time'][IQR_indices], data_pivot['screen'][IQR_indices], color='red')
#plt.scatter(data_pivot['time'][zScore_indices], data_pivot['screen'][zScore_indices], color='yellow')
plt.scatter(data_pivot['time'][common_indices], data_pivot['screen'][common_indices], color='orange')
plt.xlabel("Time")
plt.ylabel("Screen Time")
plt.title("Outliers in 'Screen'")
plt.legend(["Non-outlier points", "IQR Outliers","Both IQR and Z-Score Outliers"])
plt.show()

In [ ]:
#Grubb's Test for normally distributed data

variables = ['appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 
             'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 
             'appCat.utilities']

grubbs_results = {}
for variable in variables:
    outlier_index = grubbs.max_test_indices(data_pivot[variable], alpha=.05)
    grubbs_results[variable] = outlier_index

for variable, result in grubbs_results.items():
    print(f"Result for {variable}: {result}")